# Modelo de Regresión Multiple

Ahora consideramos una regresión con múltiples covariables o predictores:

$$y_i = \beta_0 + \beta_1 x_{1,i} + ... + \beta_{k-1} x_{k-1, i}$$

Observe que queremos estimar $k$ parámetros ($\beta_0$ incluido)

Para esto debemos (re)-escribir la ecuación anterior en forma matricial, así:

$$Y = X\beta + \varepsilon$$

El modelo coonsta de $N$ observaciones, $k$ parámetros, y $k-1$ variables

Entonces definimos cuatro matrices de la forma:

$$
Y_{[N \times 1]} = X_{[N \times K]} \, \beta_{[K \times 1]} + \varepsilon_{[N \times 1]}
$$ 

$$
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{bmatrix}
=
\begin{bmatrix}
1 & x_{11} & x_{21} & \dots & x_{k-1,1} \\
1 & x_{12} & x_{22} & \dots & x_{k-1,2} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x_{1N} & x_{2N} & \dots & x_{k-1,N}
\end{bmatrix}
\cdot
\begin{bmatrix}
\beta_0 \\
\beta_1 \\
\vdots \\
\beta_K
\end{bmatrix}
+
\begin{bmatrix}
\varepsilon_1 \\
\varepsilon_2 \\
\vdots \\
\varepsilon_n
\end{bmatrix}
$$

Tenemos de esta manera $n$ **ecuaciones**

## Supuestos de Regresión en forma Matricial

¿Cómo se ven los supuestos sobre el error cuando tenemos matrices?

$$
\varepsilon \varepsilon' =
\begin{bmatrix}
\varepsilon_1 \\
\varepsilon_2 \\
\vdots \\
\varepsilon_n
\end{bmatrix}
\begin{bmatrix}
\varepsilon_1 & \varepsilon_2 & \dots & \varepsilon_n
\end{bmatrix}
$$

$$
=
\begin{bmatrix}
\varepsilon_1^2 & \varepsilon_1\varepsilon_2 & \dots & \varepsilon_1\varepsilon_n \\
\varepsilon_2\varepsilon_1 & \varepsilon_2^2 & \dots & \varepsilon_2\varepsilon_n \\
\vdots & \vdots & \ddots & \vdots \\
\varepsilon_n\varepsilon_1 & \varepsilon_n\varepsilon_2 & \dots & \varepsilon_n^2
\end{bmatrix}
$$

$$
\Rightarrow \; E[\varepsilon \varepsilon'] = ??
$$

Observe que bajo homoscedasticidad y no autocorrelación:

$$
E[\varepsilon \varepsilon'] =
\begin{bmatrix}
\sigma^2 & 0 & \dots & 0 \\
0 & \sigma^2 & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & \sigma^2
\end{bmatrix}
= \sigma^2 I_n
$$